In [1]:
# google colab
!pip install openai python-dotenv langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00


In [4]:
# google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

# from rw_formula import RW_Calc

# google colab
import sys
sys.path.append("/content/drive/MyDrive/chat-db")
from rw_formula import RW_Calc

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [17]:
# google colab
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/chat-db/.env')

True

In [21]:
# google colab
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:////content/drive/MyDrive/chat-db/rwa_data.db") # google colab
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [22]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=False,
    handle_parsing_errors=True
)

In [23]:
agent.run("Report the total RWA and EAD per SEGMENT from RWA_122022 table")



> Entering new SQLDatabaseChain chain...
SELECT SEGMENT, SUM(RWA), SUM(EAD) FROM RWA_122022 GROUP BY SEGMENT
SQLQuery:SELECT "SEGMENT", SUM("RWA"), SUM("EAD") FROM "RWA_122022" GROUP BY "SEGMENT"
SQLResult: [('Corporate', 85075812.12498187, 156657332.34921697), ('Retail', 37806680.90001285, 165698955.81288758)]
Answer:Segment: Corporate
Total RWA: 85,075,812.12
Total EAD: 156,657,332.35

Segment: Retail
Total RWA: 37,806,680.90
Total EAD: 165,698,955.81
> Finished chain.


'The total RWA and EAD per segment from the RWA_122022 table are as follows:\n- Segment: Corporate\n  - Total RWA: 85,075,812.12\n  - Total EAD: 156,657,332.35\n- Segment: Retail\n  - Total RWA: 37,806,680.90\n  - Total EAD: 165,698,955.81'

In [24]:
agent.run("What is the sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table")



> Entering new SQLDatabaseChain chain...
SELECT ACCOUNT_NUMBER FROM RWA_122022 WHERE STATUS = 'Performing' AND ACCOUNT_NUMBER IN (SELECT ACCOUNT_NUMBER FROM RWA_032023 WHERE STATUS = 'Non Performing')
SQLQuery:SELECT "ACCOUNT_NUMBER" FROM "RWA_122022" WHERE "STATUS" = 'Performing' AND "ACCOUNT_NUMBER" IN (SELECT "ACCOUNT_NUMBER" FROM "RWA_032023" WHERE "STATUS" = 'Non Performing')
SQLResult: [('9949846894mqqb38',), ('8589346566omjm76',), ('3939009254tfzg85',), ('6572063347hjeb20',), ('8888826809jvcg55',), ('2495047535ewfk71',), ('3327992248wyhg31',), ('4271177801gmpp50',), ('4659898429miyb20',), ('4712033057jwab19',), ('9960431785dctn31',), ('2129961401ercj54',)]
Answer:There are 12 account numbers in the RWA_122022 table that have a status of 'Performing' and are also present in the RWA_032023 table with a status of 'Non Performing'.
> Finished chain.


> Entering new SQLDatabaseChain chain...
SELECT SUM(EXPOSURE) FROM RWA_122022 WHERE ACCOUNT_NUMBER IN (SELECT ACCOUNT_NUMBER FROM R

'The sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table is 575151.3173016.'

In [25]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:////content/drive/MyDrive/chat-db/rwa_data.db") # google colab
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="risk weight calculator",
        func=RW_Calc,
        description=
        """calculates a loan risk weight give the argulamts:
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'),
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
    ),
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [26]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=False,
    handle_parsing_errors=True
)

In [27]:
agent.run("Read the information for Client 'sfman997ux8036' in RWA_122022 table and then calculate the RW if PD doubles")



> Entering new SQLDatabaseChain chain...
SELECT * FROM RWA_122022 WHERE Client = 'sfman997ux8036'
SQLQuery:SELECT * FROM RWA_122022 WHERE CLIENT_NUMBER = 'sfman997ux8036'
SQLResult: [('sfman997ux8036', '1324710203qiph20', 'CREDIT_CARD', 'Performing', 'Retail', 0.0, 18728.13876, 18728.13876, 11004.65713, 0.0, 0.0246518, 0.3092, 42.58029755, 1.15616438, 'N', 'Y', 'N', 0.245013691, 2696.291657)]
Answer:CLIENT_NUMBER: sfman997ux8036
ACCOUNT_NUMBER: 1324710203qiph20
PRODUCT: CREDIT_CARD
STATUS: Performing
SEGMENT: Retail
EXPOSURE_DRAWN: 0.0
EXPOSURE_UNDRAWN: 18728.13876
EXPOSURE: 18728.13876
EAD: 11004.65713
EAD_COLL: 0.0
PD: 0.0246518
LGD: 0.3092
SIZE: 42.58029755
MATURITY: 1.15616438
F_MORTGAGE: N
F_REVOLVING: Y
F_LARGE_FIN: N
RW: 0.245013691
RWA: 2696.291657
> Finished chain.


'0.3950618648985617'